In [1]:
# import pyspoc

In [2]:
from pyspoc import Calculator, Config

octave not found, please see README
Octave not available: octave not found, please see README
Starting JVM with java class /home/hpj22/Documents/stats_for_point_clouds/py-spoc/pyspoc/lib/jidt/infodynamics.jar.


In [3]:
# import other helper libraries
import pandas as pd
import numpy as np

In [4]:
# generate a random dataframe
df = pd.DataFrame(np.random.randint(0,100,size=(100, 4)), columns=list('ABCD'))

In [6]:
# create a simple YAML config for pyss

# we first configure the 'Statistics' we want to use
# in this eg, we use the Covariance and the KendallTau correlation coefficient

# we then configure the 'Reducers' we want to use
# in this eg, we use the SingularValues and the Determinant

# for each Reducer function, we determine which Statistic to apply it to reduce

yaml_str_stat_reduce = """
Statistics:
  pyspoc.statistics.basic:
    # Covariance
    Covariance:
      schemes:
        cov:
          estimator: EmpiricalCovariance

    # Kendall Tau correlation coefficient
    KendallTau:
      schemes:
        sq:
          squared: True

        standard:
          squared: False

Reducers:
  pyspoc.reducers.basic:
    SingularValues:
      Statistics:
        - pyspoc.statistics.basic.Covariance.cov
        - pyspoc.statistics.basic.KendallTau.sq
        - pyspoc.statistics.basic.KendallTau.standard
      schemes:
        three:
          num_values: 3

    Determinant:
      Statistics:
        - pyspoc.statistics.basic.Covariance.cov
      schemes:
        scaled:
          scaled: True

        non-scaled:
          scaled: False
"""

In [7]:
# copy the randomly generated dataframe
my_dataset = df.copy()

# set a name for the config
my_config_name = "test_config_yml"

In [8]:
# create a Config class object
cfg = Config.from_yaml(my_config_name, yaml_str_stat_reduce)

Registering YAML string.
Building internal configuration.
  ✔ Module pyspoc.statistics.basic loaded successfully.
    ✔ Statistic Covariance scheme 'cov' added successfully.
    ✔ Statistic KendallTau scheme 'sq' added successfully.
    ✔ Statistic KendallTau scheme 'standard' added successfully.
  ✔ Module pyspoc.reducers.basic loaded successfully.
    ✔ Reducer SingularValues scheme 'three' added successfully.
    ✔ Reducer Determinant scheme 'scaled' added successfully.
    ✔ Reducer Determinant scheme 'non-scaled' added successfully.


In [9]:
calc = Calculator(my_dataset)

Normalising the dataset...



In [10]:
calc.compute(cfg)

Processing [None: pyspoc.statistics.basic.KendallTau.standard]: 100%|██████████| 3/3 [00:00<00:00, 214.89it/s]
Processing [None: pyspoc.reducers.basic.Determinant.non-scaled]: 100%|██████████| 3/3 [00:00<00:00, 1264.23it/s]


Calculation complete. Time taken: 0.0177s


In [11]:
calc.results.T

0
Statistic                                   Reducer                                               
pyspoc.statistics.basic.Covariance.cov      pyspoc.reducers.basic.SingularValues.three_1  1.059149
                                            pyspoc.reducers.basic.SingularValues.three_2  1.044860
                                            pyspoc.reducers.basic.SingularValues.three_3  0.922988
                                            pyspoc.reducers.basic.Determinant.scaled      1.231238
                                            pyspoc.reducers.basic.Determinant.non-scaled  0.901216
pyspoc.statistics.basic.KendallTau.sq       pyspoc.reducers.basic.SingularValues.three_1  1.003370
                                            pyspoc.reducers.basic.SingularValues.three_2  1.001936
                                            pyspoc.reducers.basic.SingularValues.three_3  0.997857
pyspoc.statistics.basic.KendallTau.standard pyspoc.reducers.basic.SingularValues.three_1  1.079557
                                            pyspoc.reducers.basic.SingularValues.three_2  1.028086
                                            pyspoc.reducers.basic.SingularValues.three_3  0.952982

In [12]:
%%writefile mymodule.py

import numpy as np
from pyspoc import ReducedStatistic
from typing import Union

class MyNewReducedStatistic(ReducedStatistic):

    def __init__(self):

        # Your initialisation code here.

        # Calling base class initialiser.
        super().__init__()

    @property
    def name(self) -> str:
        return "my_new_reducer_name"

    @property
    def identifier(self) -> str:
        return "my_new_reducer_identifier"

    @property
    def labels(self) -> list[str]:
        return ["my_new_reducer_label_1",
                "my_new_reducer_label_2",
                "my_new_reducer_label_n"]

    def compute(self, data: np.ndarray) -> Union[np.ndarray, float]:

        # Calculating your reducer as a reduced numpy array or float scalar.
        my_reducer = np.max(data)

        # Returning the result
        return my_reducer

Overwriting mymodule.py


In [13]:
from pathlib import Path
import os
# Add the parent of the current script's parent to sys.path
project_root = Path('./').resolve().parents[0]

In [14]:
new_mod_path = os.path.join(project_root,'jupyter_notebooks','mymodule.py')
print(new_mod_path)

/home/hpj22/Documents/stats_for_point_clouds/py-spoc/jupyter_notebooks/mymodule.py


In [17]:
# # Replace with your actual script, exact casing
# script = Path(new_mod_path)

# # Print it to confirm
# print("Resolved:", script)
# print("Exists:", script.exists())

In [24]:
my_dataset = my_dataset # this dataset is defined in a previous cell

my_config_dict = {'ReducedStatistics':
 {new_mod_path:
  {"my_new_reducer_identifier":
   {'schemes':
    {'scheme_1':{}}}}}}

my_config_name = "new_config"
cfg = Config.from_dict(my_config_name, my_config_dict)

Registering configuration dictionary object.
Building internal configuration.
  ✔ Module /home/hpj22/Documents/stats_for_point_clouds/py-spoc/jupyter_notebooks/mymodule.py already loaded.
  ✖ ReducedStatistic my_new_reducer_identifier could not be found. Skipping.


In [25]:
my_config_yaml = f"""
ReducedStatistics:
  {new_mod_path}:
    my_new_reducer_name:
      schemes:
        scheme_1:
"""
my_config_name = "new_config"
cfg = Config.from_yaml(my_config_name, my_config_yaml)

Registering YAML string.
Building internal configuration.
  ✔ Module /home/hpj22/Documents/stats_for_point_clouds/py-spoc/jupyter_notebooks/mymodule.py already loaded.
  ✖ ReducedStatistic my_new_reducer_name could not be found. Skipping.


In [26]:
calc = Calculator(my_dataset)
calc.compute(cfg)
results = calc.results

Normalising the dataset...


Calculation complete. Time taken: 0.0000s


In [20]:
results

0


In [25]:
# /home/hpj22/Documents/stats_for_point_clouds/py-spoc/jupyter_notebooks/mymodule.py